LB and teat model are requred for the prediction .... dont worry about those 

In [ ]:
LB = load(
    open('../model/labels', 'rb'))
teat_model = load_model('../model/model_1')


In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


cv2.boundingRect(): this function of OpenCV is used to draw an approximate rectangle around the binary image. This function is used mainly to highlight the region of interest after obtaining contours from an image.

In [ ]:
def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(
        dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(
            roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (32, 32), interpolation=cv2.INTER_CUBIC)
        thresh = thresh.astype("float32") / 255.0
        thresh = np.expand_dims(thresh, axis=-1)
        thresh = thresh.reshape(1, 32, 32, 1)
        ypred = teat_model.predict(thresh)
        ypred = LB.inverse_transform(ypred)
        [x] = ypred
        letters.append(x)
    return letters, image

#plt.imshow(image)


cv.imread():loads an image from the specified file.

cv.cvtColor():used to convert image from one colorspace to another.

cv.COLOR_BGR2GRAY: converts from BGR to GRAY.

cv.threshold(): is used to apply thresholding.

cv.THRESH_BINARY: If pixel intensity is greater than the set threshold, value set to 255, else set to 0 (black).
cv.THRESH_BINARY_INV: Inverted or Opposite case of cv2.THRESH_BINARY.

cv.dilate(): In this method the maximum pixel is calculated and is overlapped with an anchor resulting in the areas of bright regions grow in size and hence the image size increases. the size of an object in white shade or bright shade increases, while the size of an object in black shade or dark shade decreases.

cv.findContours(): finds the contours of an image.

cv.RETR_EXTERNAL: retrieves only the extreme outer contours.

cv.CHAIN_APPROX_SIMPLE: returns only the endpoints that are necessary for drawing the contour line.

cv.contourArea(): this method computes a contour area.

cv.rectangle(): used to draw rectangles on any image.

cv.THRESH_OTSU:  iterating through all the possible threshold values and calculating a measure of spread for the pixel levels each side of the threshold, i.e. the pixels that either fall in foreground or background.

cv.resize(): used to resize the image

cv.INTER_CUBIC:This uses bicubic interpolation for resizing the image. While resizing and interpolating new pixels, this method acts on the 4×4 neighboring pixels of the image. It then takes the weights average of the 16 pixels to create the new interpolated pixel.

In [ ]:
def get_word(letter):
    word = "".join(letter)
    return word
